# 応答テキストの取得

In [1]:
spark.sql(
    """
    SELECT query_model(
        'default.oci_ai_models.meta.llama-3.3-70b-instruct',
        '日本の首相は誰ですか？'
    ) AS questions
    """
).show(truncate=False)

+--------------+
|questions     |
+--------------+
|岸田文雄です。|
+--------------+



# 感情分析

In [1]:
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql import Row

review_data = [
    "これは素晴らしい製品です！",
    "サービスは素晴らしかったです。",
    "この購入には満足していません。",
    "この場所は強くおすすめします。",
    "もっと良くなるかもしれません。",
    "驚くべき体験でした！",
    "料理は美味しかったです。",
    "非常にがっかりしました。",
    "素晴らしい映画でした！",
    "平均的なパフォーマンスです。"
]

# 1. Define the schema FIRST
review_schema = StructType([StructField("review", StringType(), True)])

# 2. Create Row objects using the schema
review_row_data = [Row(review=review) for review in review_data]

# 3. NOW create the DataFrame
review_df = spark.createDataFrame(review_row_data, schema=review_schema)  # Schema is essential here.

from pyspark.sql.functions import expr

review_df.withColumn(
    "sentiment",
    expr(
        "query_model('default.oci_ai_models.meta.llama-3.3-70b-instruct', "
        "concat('回答は肯定的または否定的のみを返す必要があります。"
        "このレビューに対する感想は何ですか？: ', review))"
    )
).show(10, False)


+------------------------------+------------+
|review                        |sentiment   |
+------------------------------+------------+
|これは素晴らしい製品です！    |肯定的です。|
|サービスは素晴らしかったです。|肯定的です。|
|この購入には満足していません。|否定的です。|
|この場所は強くおすすめします。|肯定的です。|
|もっと良くなるかもしれません。|否定的です。|
|驚くべき体験でした！          |肯定的です。|
|料理は美味しかったです。      |肯定的です。|
|非常にがっかりしました。      |否定的です。|
|素晴らしい映画でした！        |肯定的です。|
|平均的なパフォーマンスです。  |肯定的です。|
+------------------------------+------------+

